In [551]:
import numpy as np

### Defining the variables
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**u** = (u<sub>1</sub>,u<sub>2</sub>) *is initialised to (4,2)*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**d** = (d<sub>1</sub>,d<sub>2</sub>) *is set to (2,1)*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**T** *is initialised to 1.0*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**&eta;** *is the tolerance level for the interior point algorithm, and is set to 10<sup>-5</sup>*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**max_iter** *is the maximum number of iterations for the interior point algorithm, and is set to 1000*

In [552]:
d = np.array([2,1])
u = np.array([4,2])
T = 1.0
m = 4
tolerance = 1e-5
max_iter = 1000

### Nash Welfare Function
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*f<sub>0</sub>(**u**)* = -(*log(u<sub>1</sub> - d<sub>1</sub>) + log(u<sub>2</sub> - d<sub>2</sub>)*)

In [553]:
def f(u):
    return -1*(np.log(u[0] - d[0]) + np.log(u[1] - d[1]))

### Log-Barrier Function
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*B(**u**)* = **&Sigma;**<sub>i=1</sub><sup>m</sup>(-*log(-f<sub>i</sub>(**u**))*)

In [554]:
def B(u):
    constraints = [
        12 - u[0] - 2*u[1],
        12 - u[1] - 2*u[0],
        u[0] - d[0],
        u[1] - d[1]
    ]
    out = 0.0
    for c in constraints:
        out += np.log(c)
    return -1*out

Using the log-barrier function, an approximate counterpart of the original objective function (Nash Welfare function) can be defined as:  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&phi;(***u***) = *f<sub>0</sub>(**u**)* + (*B(**u**)*)/**T**

In [555]:
def phi(u,t1):
    return (f(u) + (1.0/t1)*B(u))

Define the gradient of &phi;(***u***) as the 2x1 matrix of its partial derivatives w.r.t **u**:  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nabla;&phi;(***u***) = [(∂&phi;(***u***) / ∂u<sub>1</sub>) , (∂&phi;(***u***) / ∂u<sub>2</sub>)]

In [556]:
def grad_phi(u,t1):
    return np.array([
        (-1.0/(u[0]-d[0])) - (1.0/t1) * ((2.0/(12-u[0]-2*u[1])) + (2.0/(12-u[1]-2*u[0]))),
        (-1.0/(u[1]-d[1])) - (1.0/t1) * ((2.0/(12-u[0]-2*u[1])) + (2.0/(12-u[1]-2*u[0])))
    ])

Define the Hessian of &phi;(***u***) as:  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nabla; <sup>2</sup>(&phi;(***u***)) =  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**[**
    (∂<sup>2</sup>&phi;(***u***) / ∂u<sub>1</sub><sup>2</sup>) , (∂<sup>2</sup>&phi;(***u***) / ∂u<sub>1</sub>u<sub>2</sub>)  
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (∂<sup>2</sup>&phi;(***u***) / ∂u<sub>1</sub>u<sub>2</sub>) , (∂<sup>2</sup>&phi;(***u***) / ∂u<sub>2</sub><sup>2</sup>) **]**

In [557]:
def hess_phi(u,t1):
    return np.array([
        [(1.0/(u[0]-d[0])**2) + 2.0/t1*((2.0/(12-u[0]-2*u[1])**2) + (2.0/(12-u[1]-2*u[0])**2)), 
         4.0/t1*(1.0/(12-u[0]-2*u[1])**2) - 2.0/(u[0]-d[0])**2], 
        [4.0/t1*(1.0/(12-u[1]-2*u[0])**2) - 2.0/(u[1]-d[1])**2, 
         (1.0/(u[1]-d[1])**2) + 2.0/t1*((2.0/(12-u[0]-2*u[1])**2) + (2.0/(12-u[1]-2*u[0])**2))]
    ])

### Dual Optimal Variables


In [558]:
def dual_variables(u):
    return np.array([1/(12-u[0]-2*u[1]), 1/(12-u[1]-2*u[0]), 1/(u[0]-d[0]), 1/(u[1]-d[1])])

Given problem is converted into an unconstrained optimization problem with the objective function as &phi;(***u***) instead of the Nash Welfare function.  


### Newton Descent Method
For a tolerance level &eta; = 10<sup>-5</sup> and maximum iterations of 1000, the Newton Descent method is used to determine the unconstrained minimum for the optimization problem.  

**u** is updated as **u**<sub>new</sub> = **u** - *&nabla; <sup>2</sup>(&phi;(**u**))<sup>-1</sup>.&nabla;(&phi;(**u**))*  
**T** is updated as **T**<sub>new</sub> = **T**(1.0 + 1/(13&radic;&nu;)) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; where *&nu;* is a parameter associated with &phi;(***u***)


In [559]:
def newton_descent(x,t2):
  n = 1e-5
  max_iter = 1000
  for i in range(max_iter):
    
    hes = hess_phi(x,t2)

    x_new = x - np.dot(np.linalg.inv(hes),grad_phi(x,t2))

    if(np.linalg.norm(x_new - x)<=n):
      return x
  
  return x_new

In [560]:
iter = 1
while((m/T)>tolerance and iter<max_iter):
    u = newton_descent(u,T)
    T = T*(1 + 1.0/(13*np.sqrt(0.01)))
    iter += 1

dual_vars = dual_variables(u)


In [561]:
print("Initial t = 1.0\nFinal t = {:}".format(T))
print("\nNash Bargaining solution: ({:.8f},{:.8f})".format(u[0],u[1]))
print("Primal Optimal Value =  {:.8f}".format(-1*f(u)))
print("\nDual Optimal Variables:\n       f\u2081(u) = {:.4f}\n       f\u2082(u) = {:.4f}\n       f\u2083(u) = {:.4f}\n       f\u2084(u) = {:.4f}".format(dual_vars[0],dual_vars[1],dual_vars[2],dual_vars[3]))
print("\nIneqaulity Constraint Function Values:\n       f\u2081(u) = {:.4f}\n       f\u2082(u) = {:.4f}\n       f\u2083(u) = {:.4f}\n       f\u2084(u) = {:.4f}".format(12-u[0]-2*u[1],12-u[1]-2*u[0],u[0]-d[0],u[1]-d[1]))

Initial t = 1.0
Final t = 500084.19076267304

Nash Bargaining solution: (5.42729418,3.28635294)
Primal Optimal Value =  2.05872903

Dual Optimal Variables:
       f₁(u) = -15400245.7288
       f₂(u) = -0.4671
       f₃(u) = 0.2918
       f₄(u) = 0.4374

Ineqaulity Constraint Function Values:
       f₁(u) = -0.0000
       f₂(u) = -2.1409
       f₃(u) = 3.4273
       f₄(u) = 2.2864
